# NYC Taxi Demand Prediction

Trains 3 models on 100 NYC taxi clusters for comparison

**Models:**
- ConvLSTM (spatiotemporal learning)
- XGBoost (gradient boosting with features)
- AutoRegressive AR(7) (temporal baseline)

**Output:** Comparative analysis showing which model works best for each cluster type

## CELL 0: Environment Setup

In [12]:
import os
import sys
import warnings
import pickle
import gc
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical models
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller, grangercausalitytests

# Machine Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from xgboost import XGBRegressor

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import ConvLSTM2D, Conv2D, Dense, Flatten, Dropout, BatchNormalization, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Configuration
warnings.filterwarnings('ignore')
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

# Paths
INPUT_PATH = 'C:/Users/Anya/master_thesis/output'
OUTPUT_PATH = 'C:/Users/Anya/master_thesis/output/models_all_comparison'
CHECKPOINT_PATH = os.path.join(OUTPUT_PATH, 'checkpoints')

os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(CHECKPOINT_PATH, exist_ok=True)

# Hyperparameters
BATCH_SIZE = 32
N_LAGS = 24
TEST_SIZE = 0.2
VAL_SIZE = 0.1

print("="*80)
print("ENVIRONMENT CONFIGURED - ALL MODELS COMPARISON")
print("="*80)
print(f" TensorFlow version: {tf.__version__}")
print(f" Output directory: {OUTPUT_PATH}")
print(" Will train 3 models on all clusters for comparison")

ENVIRONMENT CONFIGURED - ALL MODELS COMPARISON
 TensorFlow version: 2.20.0
 Output directory: C:/Users/Anya/master_thesis/output/models_all_comparison
 Will train 3 models on all clusters for comparison


## CELL 1: Load & Prepare Data for All Clusters

In [ ]:
print("\n" + "="*80)
print("STEP 1: Loading Data for 100 Clusters")
print("="*80)

# Load raw data
data = pd.read_parquet(os.path.join(INPUT_PATH, 'taxi_data_with_clusters_full.parquet'))
print("\nRaw data loaded:")
print(f"  Shape: {data.shape}")
print(f"  Date range: {data['tpep_pickup_datetime'].min()} to {data['tpep_pickup_datetime'].max()}")
print(f"  Clusters: {data['kmeans_cluster'].nunique()}")

# Aggregate to hourly demand by cluster
data['time_period'] = data['tpep_pickup_datetime'].dt.floor('h')
demand = data.groupby(['time_period', 'kmeans_cluster']).size().reset_index(name='demand')
demand_matrix = demand.pivot(index='time_period', columns='kmeans_cluster', values='demand').fillna(0)
demand_matrix = demand_matrix.sort_index()

print("\nDemand matrix created:")
print(f"  Shape: {demand_matrix.shape}")
print(f"  All clusters: {demand_matrix.shape[1]}")
print(f"  Memory size: {demand_matrix.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# ============================================================================
# PART 0: DATA PREPARATION FOR TIME SERIES MODELING
# ============================================================================

# Extract temporal features from datetime
# Create hour, minutes, weekday, and boolean features for rush hours forp pickup
data['pickup_hour'] = data['tpep_pickup_datetime'].dt.hour
data['pickup_minutes'] = data['tpep_pickup_datetime'].dt.minute
bins = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60]
labels = ['00-04', '05-09', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', 
          '40-44', '45-49', '50-54', '55-59']
data['pickup_minute_bucket'] = pd.cut(data['pickup_minutes'], 
                            bins=bins, labels=labels, right=False, include_lowest=True)
data['pickup_weekday'] = data['tpep_pickup_datetime'].dt.weekday
data['pickup_month'] = data['tpep_pickup_datetime'].dt.month
# Same for dropoff
data['dropoff_hour'] = data['tpep_dropoff_datetime'].dt.hour
data['dropoff_minutes'] = data['tpep_dropoff_datetime'].dt.minute
data['dropoff_minute_bucket'] = pd.cut(data['dropoff_minutes'], 
                            bins=bins, labels=labels, right=False, include_lowest=True)
data['dropoff_weekday'] = data['tpep_dropoff_datetime'].dt.weekday
data['dropoff_month'] = data['tpep_dropoff_datetime'].dt.month

# Create boolean features for weekend (Saturday=5, Sunday=6)
data['is_weekend_pickup'] = data['pickup_weekday'].isin([5, 6]).astype(int)
data['is_weekend_dropoff'] = data['dropoff_weekday'].isin([5, 6]).astype(int)

# Create boolean feature for rush hours (7-9am and 5-7pm) to help capture peak traffic patterns
data['is_rush_hour_pickup'] = (
    ((data['pickup_hour'] >= 7) & (data['pickup_hour'] <= 9)) |
    ((data['pickup_hour'] >= 17) & (data['pickup_hour'] <= 19))
).astype(int)
data['is_rush_hour_dropoff'] = (
    ((data['dropoff_hour'] >= 7) & (data['dropoff_hour'] <= 9)) |
    ((data['dropoff_hour'] >= 17) & (data['dropoff_hour'] <= 19))
).astype(int)

# Calculate trip statistics for demand characterization
data['trip_duration'] = (data['tpep_dropoff_datetime'] - data['tpep_pickup_datetime']).dt.total_seconds() / 60
data['average_speed'] = data['trip_distance'] / (data['trip_duration'] / 60)

# Calculate distance from center of Manhattan (approx 40.7589, -73.9851)
manhattan_center_lat, manhattan_center_lon = 40.7589, -73.9851
data['distance_from_center_pickup'] = np.sqrt(
    (data['pickup_latitude'] - manhattan_center_lat)**2 +
    (data['pickup_longitude'] - manhattan_center_lon)**2
)
data['distance_from_center_dropoff'] = np.sqrt(
    (data['dropoff_latitude'] - manhattan_center_lat)**2 +
    (data['dropoff_longitude'] - manhattan_center_lon)**2
)

print("Feature engineering complete")

# Temporal split (70% train, 10% val, 20% test)
n = len(demand_matrix)
train_end = int(n * (1 - TEST_SIZE - VAL_SIZE))
val_end = int(n * (1 - TEST_SIZE))

train_data_all = demand_matrix.iloc[:train_end]
val_data_all = demand_matrix.iloc[train_end:val_end]
test_data_all = demand_matrix.iloc[val_end:]

print("\nTrain-Test Split:")
print(f"  Training: {len(train_data_all)} hours")
print(f"  Validation: {len(val_data_all)} hours")
print(f"  Test: {len(test_data_all)} hours")

# Save checkpoint
demand_matrix.to_pickle(os.path.join(CHECKPOINT_PATH, '01_demand_matrix.pkl'))
train_data_all.to_pickle(os.path.join(CHECKPOINT_PATH, '01_train_data.pkl'))
val_data_all.to_pickle(os.path.join(CHECKPOINT_PATH, '01_val_data.pkl'))
test_data_all.to_pickle(os.path.join(CHECKPOINT_PATH, '01_test_data.pkl'))

print("\n Checkpoint saved")


STEP 1: Loading Data for All 100 Clusters

Raw data loaded:
  Shape: (46189793, 30)
  Date range: 2015-01-01 00:00:00 to 2016-03-31 23:59:59
  Clusters: 100

Demand matrix created:
  Shape: (2928, 100)
  All clusters: 100
  Memory size: 2.26 MB

Train-Test Split:
  Training: 2049 hours
  Validation: 293 hours
  Test: 586 hours

✓ Checkpoint saved


## CELL 2: Analyze Cluster Characteristics

In [28]:
print("\n" + "="*80)
print("STEP 2: Analyze Cluster Characteristics")
print("="*80)

# Calculate statistics for each cluster
cluster_stats = pd.DataFrame({
    'cluster_id': demand_matrix.columns,
    'avg_hourly_demand': demand_matrix.mean(),
    'median_hourly_demand': demand_matrix.median(),
    'max_hourly_demand': demand_matrix.max(),
    'std_hourly_demand': demand_matrix.std(),
    'total_demand': demand_matrix.sum(),
    'sparsity_pct': (demand_matrix == 0).sum() / len(demand_matrix) * 100,
    'non_zero_hours': (demand_matrix != 0).sum()
}).sort_values('total_demand', ascending=False).reset_index(drop=True)

cluster_stats['demand_category'] = pd.cut(
    cluster_stats['avg_hourly_demand'],
    bins=[0, 20, 100, 700],
    labels=['Low', 'Medium', 'High']
)

print("\nCluster Demand Categories:")
print(f"  High-demand (>100 trips/hour): {(cluster_stats['demand_category'] == 'High').sum()} clusters")
print(f"  Medium-demand (20-100 trips/hour): {(cluster_stats['demand_category'] == 'Medium').sum()} clusters")
print(f"  Low-demand (<20 trips/hour): {(cluster_stats['demand_category'] == 'Low').sum()} clusters")

# Save statistics
cluster_stats.to_csv(os.path.join(OUTPUT_PATH, 'cluster_characteristics.csv'), index=False)
print("\nCluster characteristics saved")

print("\nTop 10 clusters:")
print(cluster_stats.head(10)[['cluster_id', 'avg_hourly_demand', 'sparsity_pct', 'demand_category']])


STEP 2: Analyze Cluster Characteristics

Cluster Demand Categories:
  High-demand (>100 trips/hour): 71 clusters
  Medium-demand (20-100 trips/hour): 21 clusters
  Low-demand (<20 trips/hour): 8 clusters

Cluster characteristics saved

Top 10 clusters:
   cluster_id  avg_hourly_demand  sparsity_pct demand_category
0          11         642.915642      0.409836            High
1          55         424.789617      0.478142            High
2           1         422.831626      0.409836            High
3          29         403.589139      0.443989            High
4          88         364.760246      0.375683            High
5          68         354.802596      0.683060            High
6          43         337.394126      0.341530            High
7          38         331.441598      0.512295            High
8           6         309.995902      0.580601            High
9          59         309.500000      0.409836            High


## CELL 3: Train XGBoost on All Clusters

In [13]:
print("\n" + "="*80)
print("MODEL 1: XGBoost Training")
print("="*80)

def create_xgboost_features(data_df, n_lags=24):
    """Create lag features for XGBoost"""
    df = data_df.copy()
    
    # Create lags
    for col in data_df.columns:
        for lag in range(1, n_lags + 1):
            df[f'{col}_lag_{lag}'] = data_df[col].shift(lag)
    
    # Rolling statistics
    for col in data_df.columns:
        df[f'{col}_rolling_mean_6'] = data_df[col].shift(1).rolling(window=6).mean()
        df[f'{col}_rolling_std_6'] = data_df[col].shift(1).rolling(window=6).std()
        df[f'{col}_rolling_mean_24'] = data_df[col].shift(1).rolling(window=24).mean()
    
    # Temporal features
    df['hour'] = df.index.hour
    df['day_of_week'] = df.index.dayofweek
    df['is_weekend'] = (df.index.dayofweek >= 5).astype(int)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    
    df = df.dropna()
    return df

# Create features
print("\nCreating XGBoost features...")
train_features = create_xgboost_features(train_data_all, N_LAGS)
val_features = create_xgboost_features(val_data_all, N_LAGS)
test_features = create_xgboost_features(test_data_all, N_LAGS)

feature_cols = [col for col in train_features.columns if col not in demand_matrix.columns]
print(f"  Total features: {len(feature_cols)}")

# Train XGBoost for each cluster
xgb_models = {}
xgb_metrics = {}
xgb_predictions = pd.DataFrame(index=test_features.index)

print(f"\nTraining {len(demand_matrix.columns)} XGBoost models...")
for i, cluster in enumerate(demand_matrix.columns):
    if i % 20 == 0:
        print(f"  Progress: {i+1}/100")
    
    model = XGBRegressor(
        n_estimators=100,
        max_depth=4,
        learning_rate=0.1,
        random_state=42,
        n_jobs=2,
        verbosity=0
    )
    
    model.fit(
        train_features[feature_cols], 
        train_features[cluster],
        verbose=False
    )
    
    xgb_models[cluster] = model
    
    # Predict
    pred = model.predict(test_features[feature_cols])
    xgb_predictions[cluster] = pred
    
    # Evaluate
    actual = test_features[cluster].values
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    mask = actual != 0
    mape = np.mean(np.abs((actual[mask] - pred[mask]) / actual[mask])) * 100 if mask.sum() > 0 else np.nan
    
    xgb_metrics[cluster] = {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

# Summary
xgb_mape_values = [m['MAPE'] for m in xgb_metrics.values() if not np.isnan(m['MAPE'])]
print("\nXGBoost Summary:")
print(f"  Mean MAPE: {np.mean(xgb_mape_values):.2f}%")
print(f"  Median MAPE: {np.median(xgb_mape_values):.2f}%")
print(f"  Std MAPE: {np.std(xgb_mape_values):.2f}%")
print(f"  Min MAPE: {np.min(xgb_mape_values):.2f}%")
print(f"  Max MAPE: {np.max(xgb_mape_values):.2f}%")

# Save
with open(os.path.join(OUTPUT_PATH, 'xgboost_models.pkl'), 'wb') as f:
    pickle.dump(xgb_models, f)
with open(os.path.join(OUTPUT_PATH, 'xgboost_metrics.pkl'), 'wb') as f:
    pickle.dump(xgb_metrics, f)

del train_features, val_features, test_features
gc.collect()

print("\n XGBoost models trained and saved")


MODEL 1: XGBoost Training

Creating XGBoost features...
  Total features: 2705

Training 100 XGBoost models...
  Progress: 1/100
  Progress: 21/100
  Progress: 41/100
  Progress: 61/100
  Progress: 81/100

XGBoost Summary (All 100 Clusters):
  Mean MAPE: 28.51%
  Median MAPE: 19.74%
  Std MAPE: 22.13%
  Min MAPE: 9.06%
  Max MAPE: 147.66%

 XGBoost models trained and saved


In [17]:
train_data_all.head()

kmeans_cluster,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
time_period,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,99.0,471.0,430.0,45.0,313.0,406.0,140.0,168.0,607.0,131.0,...,153.0,76.0,221.0,108.0,183.0,251.0,382.0,248.0,203.0,308.0
2015-01-01 01:00:00,110.0,477.0,385.0,7.0,376.0,227.0,164.0,135.0,691.0,224.0,...,267.0,2.0,338.0,145.0,157.0,145.0,325.0,276.0,271.0,227.0
2015-01-01 02:00:00,5.0,400.0,319.0,13.0,296.0,143.0,170.0,67.0,639.0,190.0,...,204.0,1.0,250.0,106.0,96.0,164.0,269.0,226.0,312.0,189.0
2015-01-01 03:00:00,20.0,273.0,454.0,19.0,219.0,86.0,319.0,30.0,577.0,157.0,...,180.0,2.0,152.0,66.0,89.0,111.0,322.0,263.0,304.0,111.0
2015-01-01 04:00:00,2.0,183.0,366.0,15.0,88.0,91.0,246.0,13.0,314.0,78.0,...,104.0,17.0,68.0,42.0,56.0,47.0,273.0,245.0,229.0,131.0


## CELL 4: Train Vector AutoRegression VAR

In [32]:
print("="*80)
print("MODEL 2: Vector AutoRegression (VAR) - Segmented by Demand Tier")
print("="*80)

# ============================================================================
# CRITICAL: VAR CURSE OF DIMENSIONALITY
# ============================================================================
# VAR(p) parameters = k^2 * p + k
# For 100 zones with p=7: 70,100 parameters on 5,000 observations → singular!
# Solution: Segment by demand tier
# ============================================================================

print("\n  NOTE: VAR on 100 clusters violates curse of dimensionality")
print("   Reason: Need k²p >> obs, but 100² × 7 = 70k params on 5k obs")
print("   Solution: Fit separate VAR models for demand tiers")

# Segment clusters by demand
high_demand_clusters = cluster_stats[cluster_stats['demand_category'] == 'High']['cluster_id'].tolist()
med_demand_clusters = cluster_stats[cluster_stats['demand_category'] == 'Medium']['cluster_id'].tolist()
low_demand_clusters = cluster_stats[cluster_stats['demand_category'] == 'Low']['cluster_id'].tolist()

print("\nCluster Segmentation:")
print(f"  High-demand (>50 trips/hr): {len(high_demand_clusters)} zones → VAR feasible")
print(f"  Medium-demand (10-50 trips/hr): {len(med_demand_clusters)} zones → AR(7) safer")
print(f"  Low-demand (<10 trips/hr): {len(low_demand_clusters)} zones → AR(3) only")

# ==============================================================
# TIER 1: VAR on High-Demand Zones (Most Spillovers)
# ==============================================================

print("\n" + "-"*80)
print("TIER 1: VAR on HIGH-DEMAND ZONES (Best signal-to-noise)")
print("-"*80)

# 1. LIMIT SCOPE: Only take top 10 zones for VAR to fix dimensionality
top_10_zones = train_data_all[high_demand_clusters].sum().nlargest(10).index.tolist()
remaining_high = list(set(high_demand_clusters) - set(top_10_zones))

# Move remaining high zones to "Medium" list (AR model)
med_demand_clusters.extend(remaining_high)
high_demand_clusters = top_10_zones

print(f"Refined Segmentation: VAR restricted to Top {len(high_demand_clusters)} zones.")

# ... inside the fitting loop ...

# 2. FORCE SMALL LAGS
# Instead of: lag_order = var_model_high.select_order(maxlags=24).aic
# Use:
lag_order = 24  # Hard constraint to prevent overfitting
print(f"  Forced Lag Order: {lag_order}")

def ensure_stationary_for_var(df, max_diff=2, p_value_threshold=0.05):
    """
    Iteratively differences the entire DataFrame until all columns are stationary 
    or max_diff is reached.
    
    Parameters:
    - df: The input DataFrame (time series data)
    - max_diff: Maximum number of differencing iterations (default 2)
    - p_value_threshold: Threshold for ADF test (default 0.05)
    
    Returns:
    - df_stationary: The processed DataFrame (differenced)
    - d: The order of differencing applied (0, 1, or 2)
    """
    df_test = df.copy()
    
    for d in range(max_diff + 1):
        non_stationary_cols = []
        
        # Check stationarity for each column
        for col in df_test.columns:
            try:
                # Drop NaNs created by differencing before passing to ADF
                series = df_test[col].dropna()
                
                # Safety check for short series
                if len(series) < 10:
                    continue
                    
                # ADF Test
                adf_result = adfuller(series, autolag='AIC')
                p_value = adf_result[1]  # p-value is the second return value
                
                if p_value > p_value_threshold:
                    non_stationary_cols.append(col)
            except Exception as e:
                # If ADF fails, assume non-stationary to be safe
                non_stationary_cols.append(col)
        
        n_non_stat = len(non_stationary_cols)
        print(f"Checking stationarity (d={d}): {n_non_stat}/{len(df_test.columns)} columns are non-stationary")
        
        # If all columns are stationary, return the current dataframe and d
        if n_non_stat == 0:
            return df_test, d
            
        # If we haven't reached the limit, difference the entire dataframe
        if d < max_diff:
            print(f"  > Differencing data (d={d} -> {d+1})...")
            df_test = df_test.diff().dropna()
            
    # If max_diff is reached, warn but return the best effort (usually d=1 or d=2)
    print(f"Warning: {n_non_stat} columns still non-stationary after d={max_diff} differencing.")
    return df_test, max_diff

if len(high_demand_clusters) > 0:
    train_var_high = train_data_all[high_demand_clusters]
    test_var_high = test_data_all[high_demand_clusters]
    
    print("\nHigh-Demand VAR:")
    print(f"  Zones: {len(high_demand_clusters)}")
    print(f"  Parameters (p=7): {len(high_demand_clusters)**2 * 7 + len(high_demand_clusters)}")
    print(f"  Obs/param ratio: {len(train_var_high) / (len(high_demand_clusters)**2 * 7 + len(high_demand_clusters)):.2f} ✓")
    
    # Stationarity check
    print("\nChecking stationarity...")
    stat_count = 0
    for col in train_var_high.columns:
        result = adfuller(train_var_high[col].dropna(), autolag='AIC')
        p_val = result[1]  # p-value is the second element (index 1)
        if p_val < 0.05:
            stat_count += 1
    print(f"  Stationary: {stat_count}/{len(train_var_high.columns)}")
    
    # Difference if needed
    train_var_high_stat, d_order_high = ensure_stationary_for_var(train_var_high)
    print(f"  Differencing order: d={d_order_high}")
    
    # Fit VAR
    try:
        print(f"\nFitting VAR on {len(high_demand_clusters)} high-demand zones...")
        var_model_high = VAR(train_var_high_stat)
        
        print(f"  Optimal lag (AIC): {lag_order}")
        
        var_results_high = var_model_high.fit(lag_order)
        
        print(f"  AIC: {var_results_high.aic:.2f}")
        print(f"  BIC: {var_results_high.bic:.2f}")
        
        # Stability
        eigenvalues = var_results_high.roots
        is_stable = np.all(np.abs(eigenvalues) < 1)
        print(f"  Stable: {'✓' if is_stable else '✗'} (max eigenvalue: {np.max(np.abs(eigenvalues)):.3f})")
        
        # Forecast
        print("\nForecasting high-demand zones...")
        var_pred_high_list = []
        last_obs = train_var_high_stat.iloc[-lag_order:].values
        
        for i in range(len(test_var_high)):
            if i % 100 == 0 and i > 0:
                print(f"  Progress: {i}/{len(test_var_high)}")
            
            forecast = var_results_high.forecast(last_obs, steps=1)
            var_pred_high_list.append(forecast[0])
            last_obs = np.vstack([last_obs[1:], forecast])
        
        var_pred_high = np.array(var_pred_high_list)
        
        # Inverse difference
        if d_order_high > 0:
            var_pred_high_undiff = var_pred_high.copy()
            for d in range(d_order_high):
                var_pred_high_undiff = np.cumsum(var_pred_high_undiff, axis=0) + train_var_high.iloc[-1].values
            var_pred_high = var_pred_high_undiff
        
        var_pred_high_df = pd.DataFrame(var_pred_high, index=test_var_high.index, columns=high_demand_clusters)
        
        # Evaluate
        var_metrics_high = {}
        var_mape_high = []
        
        for cluster in high_demand_clusters:
            actual = test_var_high[cluster].values
            pred = var_pred_high_df[cluster].values
            
            mask = actual != 0
            mape = np.mean(np.abs((actual[mask] - pred[mask]) / actual[mask])) * 100 if mask.sum() > 0 else np.nan
            
            var_metrics_high[cluster] = {
                'RMSE': np.sqrt(mean_squared_error(actual, pred)),
                'MAE': mean_absolute_error(actual, pred),
                'MAPE': mape
            }
            if not np.isnan(mape):
                var_mape_high.append(mape)
        
        print("\nHigh-Demand VAR Performance:")
        print(f"  Mean MAPE: {np.mean(var_mape_high):.2f}%")
        print(f"  Median MAPE: {np.median(var_mape_high):.2f}%")
        print(f"  Std: {np.std(var_mape_high):.2f}%")
        
        # Granger Causality
        print("\nGranger Causality (High-Demand Zones)...")
        causality_pairs = []
        
        # Test all pairs (might be slow for 20+ zones, sample if needed)
        zone_sample = high_demand_clusters[:10] if len(high_demand_clusters) > 10 else high_demand_clusters
        
        for i, effect in enumerate(zone_sample):
            for j, cause in enumerate(zone_sample):
                if i != j:
                    try:
                        gc_result = grangercausalitytests(
                            train_var_high[[effect, cause]].dropna(),
                            maxlag=7,
                            verbose=False
                        )
                        min_p = min([gc_result[lag][0][0][1] for lag in range(1, 8)])
                        if min_p < 0.05:
                            causality_pairs.append({
                                'cause': int(cause),
                                'effect': int(effect),
                                'p_value': min_p
                            })
                    except:
                        pass
        
        if causality_pairs:
            caus_df = pd.DataFrame(causality_pairs).sort_values('p_value')
            print(f"  Significant relationships (p<0.05, top 5):")
            for idx, row in caus_df.head(5).iterrows():
                print(f"    Zone {row['cause']:.0f} → Zone {row['effect']:.0f} (p={row['p_value']:.4f})")
        
        # Save
        with open(os.path.join(OUTPUT_PATH, 'var_metrics_high.pkl'), 'wb') as f:
            pickle.dump(var_metrics_high, f)
        
        var_pred_high_df.to_csv(os.path.join(OUTPUT_PATH, 'var_predictions_high.csv'))
        
        print("\n✓ High-demand VAR saved")
        
    except Exception as e:
        print(f"  Error fitting high-demand VAR: {str(e)[:100]}")
        var_metrics_high = {}
        var_mape_high = [np.nan]

# ==============================================================
# TIER 2: AR(7) on Medium-Demand Zones (Avoid VAR overfitting)
# ==============================================================

print("\n" + "-"*80)
print("TIER 2: AR(7) on MEDIUM-DEMAND ZONES (Univariate safer)")
print("-"*80)

if len(med_demand_clusters) > 0:
    print("\nMedium-demand AR(7):")
    print(f"  Zones: {len(med_demand_clusters)}")
    print(f"  Reason: VAR would have {len(med_demand_clusters)**2 * 7 + len(med_demand_clusters)} params (too many)")
    print("  Using univariate AR(7) instead (~50 params per zone)")
    
    from statsmodels.tsa.api import AutoReg
    
    var_pred_med_list = []
    var_metrics_med = {}
    var_mape_med = []
    
    print("\nFitting AR(7) for each medium-demand zone...")
    for i, cluster in enumerate(med_demand_clusters):
        if i % 10 == 0:
            print(f"  Progress: {i+1}/{len(med_demand_clusters)}")
        
        try:
            ar_model = AutoReg(train_data_all[cluster].dropna(), lags=7)
            ar_result = ar_model.fit()
            
            forecast = ar_result.get_forecast(steps=len(test_data_all)).predicted_mean.values
            var_pred_med_list.append(forecast)
            
            actual = test_data_all[cluster].values
            mask = actual != 0
            mape = np.mean(np.abs((actual[mask] - forecast[mask]) / actual[mask])) * 100 if mask.sum() > 0 else np.nan
            
            var_metrics_med[cluster] = {
                'RMSE': np.sqrt(mean_squared_error(actual, forecast)),
                'MAE': mean_absolute_error(actual, forecast),
                'MAPE': mape
            }
            if not np.isnan(mape):
                var_mape_med.append(mape)
        except:
            var_pred_med_list.append(np.full(len(test_data_all), train_data_all[cluster].mean()))
            var_metrics_med[cluster] = {'RMSE': np.nan, 'MAE': np.nan, 'MAPE': np.nan}
    
    var_pred_med_df = pd.DataFrame(
        np.array(var_pred_med_list).T,
        index=test_data_all.index,
        columns=med_demand_clusters
    )
    
    print("\nMedium-demand AR(7) Performance:")
    if var_mape_med:
        print(f"  Mean MAPE: {np.mean(var_mape_med):.2f}%")
        print(f"  Median MAPE: {np.median(var_mape_med):.2f}%")
    
    with open(os.path.join(OUTPUT_PATH, 'var_metrics_med.pkl'), 'wb') as f:
        pickle.dump(var_metrics_med, f)
    var_pred_med_df.to_csv(os.path.join(OUTPUT_PATH, 'var_predictions_med.csv'))
    
    print(" Medium-demand AR(7) saved")
else:
    var_metrics_med = {}
    var_mape_med = [np.nan]

# ==============================================================
# TIER 3: AR(3) on Low-Demand Zones (Data is sparse)
# ==============================================================

print("\n" + "-"*80)
print("TIER 3: AR(3) on LOW-DEMAND ZONES (Sparse data, simple models)")
print("-"*80)

if len(low_demand_clusters) > 0:
    print("\nLow-demand AR(3):")
    print(f"  Zones: {len(low_demand_clusters)}")
    print("  Reason: Sparse data (30%+ zeros), need simpler models")
    
    var_pred_low_list = []
    var_metrics_low = {}
    var_mape_low = []
    
    print("\nFitting AR(3) for each low-demand zone...")
    for i, cluster in enumerate(low_demand_clusters):
        if i % 20 == 0:
            print(f"  Progress: {i+1}/{len(low_demand_clusters)}")
        
        try:
            ar_model = AutoReg(train_data_all[cluster].dropna(), lags=3)
            ar_result = ar_model.fit()
            
            forecast = ar_result.get_forecast(steps=len(test_data_all)).predicted_mean.values
            var_pred_low_list.append(forecast)
            
            actual = test_data_all[cluster].values
            mask = actual != 0
            mape = np.mean(np.abs((actual[mask] - forecast[mask]) / actual[mask])) * 100 if mask.sum() > 0 else np.nan
            
            var_metrics_low[cluster] = {
                'RMSE': np.sqrt(mean_squared_error(actual, forecast)),
                'MAE': mean_absolute_error(actual, forecast),
                'MAPE': mape
            }
            if not np.isnan(mape):
                var_mape_low.append(mape)
        except:
            var_pred_low_list.append(np.full(len(test_data_all), train_data_all[cluster].mean()))
            var_metrics_low[cluster] = {'RMSE': np.nan, 'MAE': np.nan, 'MAPE': np.nan}
    
    var_pred_low_df = pd.DataFrame(
        np.array(var_pred_low_list).T,
        index=test_data_all.index,
        columns=low_demand_clusters
    )
    
    print("\nLow-demand AR(3) Performance:")
    if var_mape_low:
        print(f"  Mean MAPE: {np.mean(var_mape_low):.2f}%")
        print(f"  Median MAPE: {np.median(var_mape_low):.2f}%")
    
    with open(os.path.join(OUTPUT_PATH, 'var_metrics_low.pkl'), 'wb') as f:
        pickle.dump(var_metrics_low, f)
    var_pred_low_df.to_csv(os.path.join(OUTPUT_PATH, 'var_predictions_low.csv'))
    
    print("✓ Low-demand AR(3) saved")
else:
    var_metrics_low = {}
    var_mape_low = [np.nan]

# ==============================================================
# SUMMARY: Combine all predictions
# ==============================================================

print("\n" + "="*80)
print("SUMMARY: Segmented Temporal Models by Demand Tier")
print("="*80)

# Combine all metrics
var_metrics = {**var_metrics_high, **var_metrics_med, **var_metrics_low}

# Summary statistics
summary_dict = {
    'Tier': ['High-Demand (VAR)', 'Medium-Demand (AR7)', 'Low-Demand (AR3)'],
    'Zones': [len(high_demand_clusters), len(med_demand_clusters), len(low_demand_clusters)],
    'Model_Type': ['VAR(7)', 'AR(7) individual', 'AR(3) individual'],
    'Approx_Params': [
        len(high_demand_clusters)**2 * 7 + len(high_demand_clusters),
        len(med_demand_clusters) * 7,
        len(low_demand_clusters) * 3
    ],
    'Mean_MAPE': [
        np.mean(var_mape_high) if var_mape_high and not np.isnan(var_mape_high[0]) else np.nan,
        np.mean(var_mape_med) if var_mape_med and not np.isnan(var_mape_med[0]) else np.nan,
        np.mean(var_mape_low) if var_mape_low and not np.isnan(var_mape_low[0]) else np.nan
    ]
}

summary_var_df = pd.DataFrame(summary_dict)
summary_var_df.to_csv(os.path.join(OUTPUT_PATH, 'var_summary_by_tier.csv'), index=False)

print("\n" + summary_var_df.to_string(index=False))

print("\n All segmented models trained and saved")
print(f"  ('segmented approach: VAR on n={len(high_demand_clusters)} high-demand zones')")
print("  ('(80% of trips), AR(7) on medium-demand, and AR(3) on sparse low-demand zones.')")
print("  ('This methodology balances theoretical rigor with practical constraints.')")

MODEL 2: Vector AutoRegression (VAR) - Segmented by Demand Tier

  NOTE: VAR on 100 clusters violates curse of dimensionality
   Reason: Need k²p >> obs, but 100² × 7 = 70k params on 5k obs
   Solution: Fit separate VAR models for demand tiers

Cluster Segmentation:
  High-demand (>50 trips/hr): 71 zones → VAR feasible
  Medium-demand (10-50 trips/hr): 21 zones → AR(7) safer
  Low-demand (<10 trips/hr): 8 zones → AR(3) only

--------------------------------------------------------------------------------
TIER 1: VAR on HIGH-DEMAND ZONES (Best signal-to-noise)
--------------------------------------------------------------------------------
Refined Segmentation: VAR restricted to Top 10 zones.
  Forced Lag Order: 24

High-Demand VAR:
  Zones: 10
  Parameters (p=7): 710
  Obs/param ratio: 2.89 ✓

Checking stationarity...
  Stationary: 10/10
Checking stationarity (d=0): 0/10 columns are non-stationary
  Differencing order: d=0

Fitting VAR on 10 high-demand zones...
  Optimal lag (AIC): 24

## CELL 5: Train ConvLSTM

In [30]:
print("\n" + "="*80)
print("MODEL 3: ConvLSTM Training")
print("="*80)

# Select top 20 high-demand clusters for ConvLSTM
high_demand_clusters = cluster_stats.head(20)['cluster_id'].tolist()

print(f"\nConvLSTM on {len(high_demand_clusters)} high-demand clusters")
print(f"  Reason: ConvLSTM computationally expensive, best on rich data")

# Prepare data
train_convlstm = train_data_all[high_demand_clusters]
val_convlstm = val_data_all[high_demand_clusters]
test_convlstm = test_data_all[high_demand_clusters]

# Scale
scaler = MinMaxScaler()
train_scaled = pd.DataFrame(
    scaler.fit_transform(train_convlstm),
    index=train_convlstm.index,
    columns=train_convlstm.columns
)
val_scaled = pd.DataFrame(
    scaler.transform(val_convlstm),
    index=val_convlstm.index,
    columns=val_convlstm.columns
)
test_scaled = pd.DataFrame(
    scaler.transform(test_convlstm),
    index=test_convlstm.index,
    columns=test_convlstm.columns
)

# Create grid (5x4 = 20 clusters)
def create_grid_data(data_df, grid_height=5, grid_width=4):
    n_timesteps = len(data_df)
    grid_data = np.zeros((n_timesteps, grid_height, grid_width, 1))
    for idx, col in enumerate(data_df.columns):
        row = idx // grid_width
        col_pos = idx % grid_width
        if row < grid_height:
            grid_data[:, row, col_pos, 0] = data_df[col].values
    return grid_data

grid_train = create_grid_data(train_scaled, 5, 4)
grid_val = create_grid_data(val_scaled, 5, 4)
grid_test = create_grid_data(test_scaled, 5, 4)

print(f"\nGrid shapes: {grid_train.shape}")

# Build ConvLSTM model
input_layer = Input(shape=(6, 5, 4, 1), name='input')
x = ConvLSTM2D(32, (3, 3), padding='same', return_sequences=True, activation='relu')(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)
x = ConvLSTM2D(32, (3, 3), padding='same', return_sequences=False, activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(16, (3, 3), padding='same', activation='relu')(x)
output = Conv2D(1, (1, 1), padding='same', activation='relu', name='output')(x)

convlstm_model = Model(inputs=input_layer, outputs=output)
convlstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

print("\nTraining ConvLSTM...")
history = convlstm_model.fit(
    grid_train, grid_train,
    validation_data=(grid_val, grid_val),
    epochs=50,
    batch_size=BATCH_SIZE,
    callbacks=[
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=0),
    ],
    verbose=0
)

# Predict
convlstm_pred = convlstm_model.predict(grid_test, verbose=0)

# Extract and evaluate
convlstm_metrics = {}
convlstm_predictions = pd.DataFrame(index=test_convlstm.index)

for idx, cluster in enumerate(high_demand_clusters):
    row = idx // 4
    col = idx % 4
    
    pred_grid = convlstm_pred[:, row, col, 0]
    convlstm_predictions[cluster] = scaler.inverse_transform(
        np.column_stack([pred_grid] * len(train_convlstm.columns))
    )[:, 0]
    
    actual = test_convlstm[cluster].values
    rmse = np.sqrt(mean_squared_error(actual, convlstm_predictions[cluster].values))
    mae = mean_absolute_error(actual, convlstm_predictions[cluster].values)
    mask = actual != 0
    mape = np.mean(np.abs((actual[mask] - convlstm_predictions[cluster].values[mask]) / actual[mask])) * 100 if mask.sum() > 0 else np.nan
    
    convlstm_metrics[cluster] = {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

# Summary
convlstm_mape_values = [m['MAPE'] for m in convlstm_metrics.values() if not np.isnan(m['MAPE'])]
print(f"\nConvLSTM Summary ({len(high_demand_clusters)} High-Demand Clusters):")
print(f"  Mean MAPE: {np.mean(convlstm_mape_values):.2f}%")
print(f"  Median MAPE: {np.median(convlstm_mape_values):.2f}%")

# Save
convlstm_model.save(os.path.join(OUTPUT_PATH, 'convlstm_model.keras'))
with open(os.path.join(OUTPUT_PATH, 'convlstm_metrics.pkl'), 'wb') as f:
    pickle.dump(convlstm_metrics, f)

print("\n ConvLSTM model trained and saved")


MODEL 3: ConvLSTM Training

ConvLSTM on 20 high-demand clusters
  Reason: ConvLSTM computationally expensive, best on rich data

Grid shapes: (2049, 5, 4, 1)

Training ConvLSTM...


ValueError: Input 0 of layer "functional_2" is incompatible with the layer: expected shape=(None, 6, 5, 4, 1), found shape=(None, 5, 4)

## CELL 6: Comparative Analysis & Model Performance Ranking

In [ ]:
print("\n" + "="*80)
print("COMPARATIVE ANALYSIS: Which Model Works Best for Each Cluster?")
print("="*80)

# Create comparison dataframe
comparison_data = []

for cluster in demand_matrix.columns:
    row = {'cluster_id': cluster}
    
    # Get demand category
    cluster_info = cluster_stats[cluster_stats['cluster_id'] == cluster].iloc[0]
    row['avg_demand'] = cluster_info['avg_hourly_demand']
    row['demand_category'] = cluster_info['demand_category']
    row['sparsity_pct'] = cluster_info['sparsity_pct']
    
    # XGBoost metrics
    if cluster in xgb_metrics:
        row['xgboost_mape'] = xgb_metrics[cluster]['MAPE']
    else:
        row['xgboost_mape'] = np.nan
    
    # AR(7) metrics
    if cluster in ar_metrics:
        row['ar7_mape'] = ar_metrics[cluster]['MAPE']
    else:
        row['ar7_mape'] = np.nan
    
    # ConvLSTM metrics (only for top 20 clusters)
    if cluster in convlstm_metrics:
        row['convlstm_mape'] = convlstm_metrics[cluster]['MAPE']
    else:
        row['convlstm_mape'] = np.nan
    
    # Determine best model
    mapes = {
        'XGBoost': row['xgboost_mape'],
        'AR(7)': row['ar7_mape'],
        'ConvLSTM': row['convlstm_mape']
    }
    # Remove NaN values
    mapes = {k: v for k, v in mapes.items() if not np.isnan(v)}
    
    if mapes:
        row['best_model'] = min(mapes, key=mapes.get)
        row['best_mape'] = min(mapes.values())
    else:
        row['best_model'] = 'N/A'
        row['best_mape'] = np.nan
    
    comparison_data.append(row)

comparison_df = pd.DataFrame(comparison_data)

# Save comparison
comparison_df.to_csv(os.path.join(OUTPUT_PATH, 'model_comparison_all_clusters.csv'), index=False)

print(f"\nComparison Results:")
print(f"\nBest Model Distribution:")
print(comparison_df['best_model'].value_counts())

print(f"\nPerformance by Model (MAPE):")
print(f"  XGBoost - Mean: {comparison_df['xgboost_mape'].mean():.2f}%, Median: {comparison_df['xgboost_mape'].median():.2f}%")
print(f"  AR(7)   - Mean: {comparison_df['ar7_mape'].mean():.2f}%, Median: {comparison_df['ar7_mape'].median():.2f}%")
print(f"  ConvLSTM - Mean: {comparison_df['convlstm_mape'].mean():.2f}%, Median: {comparison_df['convlstm_mape'].median():.2f}%")

print(f"\nBest Model by Demand Category:")
for category in ['High', 'Medium', 'Low']:
    subset = comparison_df[comparison_df['demand_category'] == category]
    if len(subset) > 0:
        best_counts = subset['best_model'].value_counts()
        print(f"  {category}: {dict(best_counts)}")

print(f"\n✓ Comparison saved to model_comparison_all_clusters.csv")

## CELL 7: Summary Tables & Insights

In [ ]:
print("\n" + "="*80)
print("FINAL SUMMARY & INSIGHTS")
print("="*80)

# Overall statistics
summary_stats = {
    'Model': ['XGBoost', 'AR(7)', 'ConvLSTM (High-Demand Only)'],
    'Clusters_Trained': [100, 100, 20],
    'Mean_MAPE': [
        comparison_df['xgboost_mape'].mean(),
        comparison_df['ar7_mape'].mean(),
        comparison_df['convlstm_mape'].mean()
    ],
    'Median_MAPE': [
        comparison_df['xgboost_mape'].median(),
        comparison_df['ar7_mape'].median(),
        comparison_df['convlstm_mape'].median()
    ],
    'Std_MAPE': [
        comparison_df['xgboost_mape'].std(),
        comparison_df['ar7_mape'].std(),
        comparison_df['convlstm_mape'].std()
    ],
    'Best_for_Count': [
        (comparison_df['best_model'] == 'XGBoost').sum(),
        (comparison_df['best_model'] == 'AR(7)').sum(),
        (comparison_df['best_model'] == 'ConvLSTM').sum()
    ]
}

summary_df = pd.DataFrame(summary_stats)
summary_df.to_csv(os.path.join(OUTPUT_PATH, 'model_performance_summary.csv'), index=False)

print("\n" + summary_df.to_string(index=False))

# Insights
print(f"\n" + "="*80)
print("KEY INSIGHTS FOR THESIS CONCLUSIONS")
print("="*80)

xgb_best = (comparison_df['best_model'] == 'XGBoost').sum()
ar_best = (comparison_df['best_model'] == 'AR(7)').sum()
lstm_best = (comparison_df['best_model'] == 'ConvLSTM').sum()

print(f"\n1. OVERALL WINNER:")
if xgb_best > ar_best and xgb_best > lstm_best:
    print(f"   XGBoost is best for {xgb_best} clusters ({xgb_best/100*100:.1f}%)")
elif ar_best > xgb_best and ar_best > lstm_best:
    print(f"   AR(7) is best for {ar_best} clusters ({ar_best/100*100:.1f}%)")
else:
    print(f"   ConvLSTM is best for {lstm_best} clusters ({lstm_best/100*100:.1f}%)")

print(f"\n2. MODEL SPECIALIZATION:")
high_dem = comparison_df[comparison_df['demand_category'] == 'High']
med_dem = comparison_df[comparison_df['demand_category'] == 'Medium']
low_dem = comparison_df[comparison_df['demand_category'] == 'Low']

if len(high_dem) > 0:
    print(f"   High-demand zones: {high_dem['best_model'].value_counts().to_dict()}")
if len(med_dem) > 0:
    print(f"   Medium-demand zones: {med_dem['best_model'].value_counts().to_dict()}")
if len(low_dem) > 0:
    print(f"   Low-demand zones: {low_dem['best_model'].value_counts().to_dict()}")

print(f"\n3. SPARSE VS DENSE DATA:")
sparse = comparison_df[comparison_df['sparsity_pct'] > 30]
dense = comparison_df[comparison_df['sparsity_pct'] <= 30]
if len(sparse) > 0:
    print(f"   Sparse zones (>30% zeros): {sparse['best_model'].value_counts().to_dict()}")
if len(dense) > 0:
    print(f"   Dense zones (<=30% zeros): {dense['best_model'].value_counts().to_dict()}")

print(f"\n4. PERFORMANCE RANGE:")
print(f"   XGBoost MAPE: {comparison_df['xgboost_mape'].min():.2f}% - {comparison_df['xgboost_mape'].max():.2f}%")
print(f"   AR(7) MAPE: {comparison_df['ar7_mape'].min():.2f}% - {comparison_df['ar7_mape'].max():.2f}%")
print(f"   ConvLSTM MAPE: {comparison_df['convlstm_mape'].min():.2f}% - {comparison_df['convlstm_mape'].max():.2f}%")

print(f"\n✓ Summary saved")

## CELL 8: Visualizations

In [ ]:
print("\nCreating visualizations...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Model Performance Comparison
ax = axes[0, 0]
models = ['XGBoost', 'AR(7)', 'ConvLSTM']
means = [
    comparison_df['xgboost_mape'].mean(),
    comparison_df['ar7_mape'].mean(),
    comparison_df['convlstm_mape'].mean()
]
stds = [
    comparison_df['xgboost_mape'].std(),
    comparison_df['ar7_mape'].std(),
    comparison_df['convlstm_mape'].std()
]
ax.bar(models, means, yerr=stds, capsize=10, color=['#3498db', '#e74c3c', '#2ecc71'])
ax.set_ylabel('Mean MAPE (%)', fontsize=12)
ax.set_title('Model Performance Comparison (Mean ± Std)', fontsize=13, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# 2. Best Model Distribution
ax = axes[0, 1]
best_counts = comparison_df['best_model'].value_counts()
colors_dict = {'XGBoost': '#3498db', 'AR(7)': '#e74c3c', 'ConvLSTM': '#2ecc71'}
colors = [colors_dict.get(idx, '#95a5a6') for idx in best_counts.index]
ax.bar(best_counts.index, best_counts.values, color=colors)
ax.set_ylabel('Number of Clusters', fontsize=12)
ax.set_title('Best Model Distribution (Which Model Wins for Each Cluster)', fontsize=13, fontweight='bold')
for i, v in enumerate(best_counts.values):
    ax.text(i, v + 0.5, str(v), ha='center', fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# 3. Performance by Demand Category
ax = axes[1, 0]
categories = ['High', 'Medium', 'Low']
xgb_means = []
ar_means = []
lstm_means = []
for cat in categories:
    subset = comparison_df[comparison_df['demand_category'] == cat]
    if len(subset) > 0:
        xgb_means.append(subset['xgboost_mape'].mean())
        ar_means.append(subset['ar7_mape'].mean())
        lstm_means.append(subset['convlstm_mape'].mean())

x = np.arange(len(categories))
width = 0.25
ax.bar(x - width, xgb_means, width, label='XGBoost', color='#3498db')
ax.bar(x, ar_means, width, label='AR(7)', color='#e74c3c')
ax.bar(x + width, lstm_means, width, label='ConvLSTM', color='#2ecc71')
ax.set_ylabel('Mean MAPE (%)', fontsize=12)
ax.set_title('Performance by Demand Category', fontsize=13, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(categories)
ax.legend()
ax.grid(axis='y', alpha=0.3)

# 4. MAPE Distribution Box Plot
ax = axes[1, 1]
data_to_plot = [
    comparison_df['xgboost_mape'].dropna(),
    comparison_df['ar7_mape'].dropna(),
    comparison_df['convlstm_mape'].dropna()
]
bp = ax.boxplot(data_to_plot, labels=['XGBoost', 'AR(7)', 'ConvLSTM'], patch_artist=True)
colors = ['#3498db', '#e74c3c', '#2ecc71']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
ax.set_ylabel('MAPE (%)', fontsize=12)
ax.set_title('MAPE Distribution by Model', fontsize=13, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PATH, 'model_comparison_visualization.png'), dpi=300, bbox_inches='tight')
print("✓ Visualizations saved")
plt.show()

print(f"\n" + "="*80)
print(f"ALL ANALYSIS COMPLETE")
print(f"="*80)
print(f"\nOutput files saved to: {OUTPUT_PATH}")
print(f"\nKey files:")
print(f"  - model_comparison_all_clusters.csv (cluster-by-cluster comparison)")
print(f"  - model_performance_summary.csv (overall statistics)")
print(f"  - cluster_characteristics.csv (demand analysis)")
print(f"  - model_comparison_visualization.png (charts)")